<a href="https://colab.research.google.com/github/ashupednekar/StyleEncrypt/blob/master/Style_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
from random import randint as ri
from matplotlib import pyplot as plt

In [ ]:
#downloading fonts
!mkdir fonts
%cd fonts
!wget https://noto-website-2.storage.googleapis.com/pkgs/Noto-unhinted.zip
!unzip Noto-unhinted.zip
%cd ..

mkdir: cannot create directory ‘fonts’: File exists
/content/fonts
--2020-09-20 11:01:08--  https://noto-website-2.storage.googleapis.com/pkgs/Noto-unhinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1114561068 (1.0G) [application/zip]
Saving to: ‘Noto-unhinted.zip.1’

Noto-unhinted.zip.1 100%[===================>]   1.04G   178MB/s    in 6.6s    

2020-09-20 11:01:15 (162 MB/s) - ‘Noto-unhinted.zip.1’ saved [1114561068/1114561068]

Archive:  Noto-unhinted.zip
replace LICENSE_OFL.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
from random import choice

color_mode = 'white' # or random

def create_image_for_text(text, font):
    """Creating content image for a given input"""
    if color_mode == 'random':
        #Creating background with random color
        img = Image.new('RGB', (2048, 512), color = (ri(0,255), ri(0,255), ri(0,255)))
    else:
        #Creating background with random color
        img = Image.new('RGB', (2048, 512), color = (255, 255, 255))
    plt.figure()
    plt.imshow(img)
    fnt = ImageFont.truetype(font, 400)
    d = ImageDraw.Draw(img)
    d.text((0,0), text, font=fnt, fill=(ri(0,255), ri(0,255), ri(0,255)))
    plt.imshow(img)
    img.save('_'.join(text.split(' ')) + '.jpg')
    return '_'.join(text.split(' ')) + '.jpg'

In [ ]:
!pip install pycryptodome

In [ ]:
"""AES"""
from Crypto.Cipher import AES
from Crypto import Random

key = Random.new().read(AES.block_size)
iv = Random.new().read(AES.block_size)

class ImageEncryptor:

    def encrypt(self, path):
        input_file = open(path, 'rb')
        self.path = path
        input_data = input_file.read()
        input_file.close()
        cfb_cipher = AES.new(key, AES.MODE_CFB, iv)
        enc_data = cfb_cipher.encrypt(input_data)
        return enc_data

    def decrypt(self, enc):
        cfb_decipher = AES.new(key, AES.MODE_CFB, iv)
        plain_data = cfb_decipher.decrypt(enc)
        output_file = open(self.path, "wb")
        output_file.write(plain_data)
        output_file.close()

In [ ]:
fontface = '/content/fonts/' + choice(os.listdir('/content/fonts/'))
print(fontface)
test_img_path = create_image_for_text('Hello', fontface)

img_encryptor = ImageEncryptor()
enc = img_encryptor.encrypt(test_img_path)
img_encryptor.decrypt(enc)
plt.imshow(Image.open(test_img_path))

img_encryptor = ImageEncryptor()
enc = img_encryptor.encrypt(test_img_path)
print(enc)
os.system('rm '+test_img_path)

In [106]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools


import tensorflow as tf
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os

def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)

def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

@tf.function()
def train_step(image, loss_function):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = loss_function(outputs)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [107]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

In [108]:
class Stylizer:

    def __init__(self, content_path, style_url):
        self.content_image = self.load_img(content_path)
        self.style_image = self.load_img(self.get_style_image(style_url))
        plt.subplot(1, 2, 1)
        imshow(content_image, 'Content Image')
        plt.subplot(1, 2, 2)
        imshow(style_image, 'Style Image')
        self.preprocess(content_image)
        self.extract()
        self.stylize()
    
    def get_style_image(self, url):
        os.system('wget '+url)
        os.rename(url.split('/')[-1], 'style.jpg')
        return 'style.jpg'

    def load_img(self, path_to_img):
        max_dim = 512
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)

        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img

    def preprocess(self, content_image):
        x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
        x = tf.image.resize(x, (224, 224))
        vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
        prediction_probabilities = vgg(x)
        prediction_probabilities.shape

        predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
        print([(class_name, prob) for (number, class_name, prob) in predicted_top_5])

        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        content_layers = ['block5_conv2'] 

        style_layers = ['block1_conv1',
                        'block2_conv1',
                        'block3_conv1', 
                        'block4_conv1', 
                        'block5_conv1']

        num_content_layers = len(content_layers)
        num_style_layers = len(style_layers)

    def vgg_layers(self, layer_names):
        """ Creates a vgg model that returns a list of intermediate output values."""
        # Load our model. Load pretrained VGG, trained on imagenet data
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False
        
        outputs = [vgg.get_layer(name).output for name in layer_names]

        model = tf.keras.Model([vgg.input], outputs)
        return model

    def gram_matrix(self, input_tensor):
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)

    def extract(self):
        style_extractor = vgg_layers(style_layers)
        style_outputs = style_extractor(style_image*255)

        #Look at the statistics of each layer's output
        for name, output in zip(style_layers, style_outputs):
            print(name)
            print("  shape: ", output.numpy().shape)
            print("  min: ", output.numpy().min())
            print("  max: ", output.numpy().max())
            print("  mean: ", output.numpy().mean())
            print()

        extractor = StyleContentModel(style_layers, content_layers)

        results = extractor(tf.constant(content_image))

        print('Styles:')
        for name, output in sorted(results['style'].items()):
            print("  ", name)
            print("    shape: ", output.numpy().shape)
            print("    min: ", output.numpy().min())
            print("    max: ", output.numpy().max())
            print("    mean: ", output.numpy().mean())
            print()

        print("Contents:")
        for name, output in sorted(results['content'].items()):
            print("  ", name)
            print("    shape: ", output.numpy().shape)
            print("    min: ", output.numpy().min())
            print("    max: ", output.numpy().max())
            print("    mean: ", output.numpy().mean())

        style_targets = extractor(style_image)['style']
        content_targets = extractor(content_image)['content']

    def stylize(self):
        image = tf.Variable(content_image)
        opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
        style_weight=1e-2
        content_weight=1e4
        from tqdm import tqdm
        for i in tqdm(range(1000)):
            train_step(image, self.add_style_content_loss)
        tensor_to_image(image)

    def add_style_content_loss(self, outputs):
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                            for name in style_outputs.keys()])
        style_loss *= style_weight / num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                                for name in content_outputs.keys()])
        content_loss *= content_weight / num_content_layers
        loss = style_loss + content_loss
        return loss


In [ ]:
s = Stylizer('content.jpg', 'https://cdnb.artstation.com/p/assets/images/images/006/710/803/large/nirmala-handapangoda-nyh-wallpaper.jpg')

[('totem_pole', 0.25286594), ('bow', 0.10808957), ('pole', 0.07290625), ('electric_fan', 0.043226697), ('cowboy_boot', 0.037619915)]
block1_conv1
  shape:  (1, 313, 512, 64)
  min:  0.0
  max:  607.45654
  mean:  41.924435

block2_conv1
  shape:  (1, 156, 256, 128)
  min:  0.0
  max:  4664.601
  mean:  152.09196

block3_conv1
  shape:  (1, 78, 128, 256)
  min:  0.0
  max:  9972.932
  mean:  154.09343

block4_conv1
  shape:  (1, 39, 64, 512)
  min:  0.0
  max:  19965.637
  mean:  539.13336

block5_conv1
  shape:  (1, 19, 32, 512)
  min:  0.0
  max:  2994.7485
  mean:  37.51554





  0%|          | 0/1000 [00:00<?, ?it/s]

Styles:
   block1_conv1
    shape:  (1, 64, 64)
    min:  0.026819006
    max:  93394.83
    mean:  976.59674

   block2_conv1
    shape:  (1, 128, 128)
    min:  0.0
    max:  209931.33
    mean:  26442.84

   block3_conv1
    shape:  (1, 256, 256)
    min:  0.0
    max:  900648.75
    mean:  42511.11

   block4_conv1
    shape:  (1, 512, 512)
    min:  0.0
    max:  8395350.0
    mean:  459386.5

   block5_conv1
    shape:  (1, 512, 512)
    min:  0.0
    max:  307541.12
    mean:  3454.8445

Contents:
   block5_conv2
    shape:  (1, 18, 32, 512)
    min:  0.0
    max:  1851.0177
    mean:  19.496836




  0%|          | 1/1000 [00:00<12:43,  1.31it/s]

  0%|          | 2/1000 [00:01<10:46,  1.54it/s]

  0%|          | 3/1000 [00:01<09:26,  1.76it/s]

  0%|          | 4/1000 [00:01<08:28,  1.96it/s]



  0%|          | 5/1000 [00:02<07:48,  2.12it/s]

...



  6%|▌         | 62/1000 [00:27<06:40,  2.34it/s]



  6%|▋         | 63/1000 [00:27<06:37,  2.36it/s]



  6%|▋         | 64/1000 [00:28<06:35,  2.36it/s]



  6%|▋         | 65/1000 [00:28<06:32,  2.38it/s]



  7%|▋         | 66/1000 [00:28<06:29,  2.40it/s]